# Generic Fold
---

In our last notebook, we wrote a generic method (`abstraction`) which represents general recursive computation on List structure. We then used it to write other more concrete computations (`sum`, `product` and `length`). The benefit of writing such a general method is obvious, less repetitive code and less code to debug. Let's say we want to use the `abstraction` method to implement the following `double` method.

In [1]:
sealed trait IntList
final case class IntPair(element: Int, rest: IntList) extends IntList
final case object IntEnd extends IntList

object IntList {
    def double(list: IntList): IntList = list match {
        case IntEnd => IntEnd
        case IntPair(first, rest) => IntPair(first * 2, double(rest))
    }
}

val list: IntList = IntPair(1, IntPair(2, IntPair(3, IntPair(4, IntEnd))))

IntList.double(list)

defined trait IntList
defined class IntPair
defined object IntEnd
defined object IntList
list: IntList = IntPair(1, IntPair(2, IntPair(3, IntPair(4, IntEnd))))
res0_5: IntList = IntPair(2, IntPair(4, IntPair(6, IntPair(8, IntEnd))))

### connecting Lego blocks
---

Composing expressions in typed language is much like connecting the right blocks of Lego toy together. Let's say if you want to compose two functions to produce a "composed" function, the important invariant is, the input type of the second function should match the output type of the first function. Let's try it out.

In [2]:
def thirdPart(x: Int): Double = x / 3.0

def doubleIt(x: Double): Double = x * 2

def doubleOfThirdPart(x: Int): Double = doubleIt(thirdPart(x))

doubleOfThirdPart(10) // (10 / 3) * 2

defined function thirdPart
defined function doubleIt
defined function doubleOfThirdPart
res1_3: Double = 6.666666666666667

The result of `thirdPart` returns `Double` value, and `doubleIt` expects `Double` as an input. That is why we can compose them and write a method which takes `Int` and returns `Double` value.

Let's see if we can write `double` (of `IntList`) using the `abstraction` method.

In [3]:
def abstraction(list: IntList, endCase: Int, pairCase: (Int, Int) => Int): Int = list match {
    case IntEnd => endCase
    case IntPair(first, rest) => pairCase(first, abstraction(rest, endCase, pairCase))
}

defined function abstraction

Now, looking at the implementation of `double`, let's figure out what parts we need to pass into the `abstraction` method.
The end case of `double` needs to return `End`, and the pair case needs to create a `Pair` using the head and result of the rest of the list.

Do you see the problem?

The `abstraction` method takes `Int` and `(Int, Int) => Int` for end case and pair case, respectively. And `double` doesn't use those types of value at those positions. Also, `double` returns `IntList` and `abstraction` method returns `Int`. Obviously, we can't use `abstraction` as it is. We need to change the signature of it, and make it a bit more generic.

### update `abstraction` to be generic in its return type
---

Rather than returning an `Int` value, we want to return a valeu of generic type, so user of this method can use to compute any recursive operation on a List. Let's break it down.

```scala
def abstraction[T](list: IntList, endCase: ???, pairCase: ???): T = list match {
    case IntEnd => ???
    case IntPair(first, rest) => ???
}
```

At the moment, we have following information about this method:
1) it takes `IntList`
2) it returns a value of some type `T`

Because it returns a value of type `T`, both cases should hold an expression which returns a type `T`. And based on that, we will figure out the types of `endCase` and `pairCase` parameters.
1) The `End` case returns `endCase` as it is (at least so far), and because the cases must return `T`, the type of `endCase` should be a `T`.
2) The `Pair` case uses a function supplied in `pairCase`, and because it uses `first` element of the List (which is `Int`), recursive call of `abstraction` method (which means `T`) and cases must return type `T` which means function's return type should be a `T`.

Let's update `abstraction` based on our reasoning and try to use it to implement `double`.

In [4]:
def abstraction[T](list: IntList, endCase: T, pairCase: (Int, T) => T): T = list match {
    case IntEnd => endCase
    case IntPair(first, rest) => pairCase(first, abstraction(rest, endCase, pairCase))
}

object IntList {
    def double(list: IntList): IntList = abstraction(list, IntEnd, (a, b) => IntPair(a * 2, b))
}

IntList.double(list)

defined function abstraction
defined object IntList
res3_2: IntList = IntPair(2, IntPair(4, IntPair(6, IntPair(8, IntEnd))))

### generic fold
---

The `abstraction` method we defined is known as the `fold` method. We use it to "fold" on some list-like structure to produce a result using recursion. The only drawback of `abstraction` is that it can only handle List of integers. Let's rename it to `fold` and implement it on a generic `List` data structure.

In [5]:
sealed trait List[A]
final case class End[A]() extends List[A]
final case class Pair[A](head: A, tail: List[A]) extends List[A]

object List {
    def fold[A, T](list: List[A], endCase: T, pairCase: (A, T) => T): T = list match {
        case End() => endCase
        case Pair(first, rest) => pairCase(first, fold(rest, endCase, pairCase))
    }
}

val listInt: List[Int] = Pair(1, Pair(2, Pair(3, Pair(4, End[Int]))))
val listString: List[String] = Pair("hi", Pair("there", Pair("how", Pair("are", Pair("you?", End[String])))))

defined trait List
defined class End
defined class Pair
defined object List
listInt: List[Int] = Pair(1, Pair(2, Pair(3, Pair(4, End()))))
listString: List[String] = Pair(
  "hi",
  Pair("there", Pair("how", Pair("are", Pair("you?", End()))))
)

The implementation of fold is pretty straightforward. Rather than taking `Int` as the first element of list we are using type `A`, because now List holds elements of type `A`.

In [6]:
List.fold(listInt, 0, (a: Int, b: Int) => a + b)

res5: Int = 10

In [7]:
List.fold(listString, "", (a: String, b: String) => a + " " + b)

res6: String = "hi there how are you? "

Notice, that we explicitly annotated type input types in function we passed to `fold`. What happens if we don't do that?

In [7]:
List.fold(listInt, 0, (a, b) => a + b)

cmd7.sc:1: missing parameter type
val res7 = List.fold(listInt, 0, (a, b) => a + b)
                                  ^cmd7.sc:1: missing parameter type
val res7 = List.fold(listInt, 0, (a, b) => a + b)
                                     ^Compilation Failed

: 

Hm.. the compiler complains that it can't figure out the types of inputs. Strenge, isn't it? Because it seems obvious based on other parameters.

### function tricks
---

Scala provides following three things to write more concise programs and helping type system infer types alongwith.
1) placeholder syntax
2) method to function conversion
3) multiple parameter lists

#### placeholder syntax
---

We already know that when we want to ignore any value, we use `_`. And if we are passing a function as a parameter, we can use `_` to denote input value. For example, `(a, b) => a + b` could be written as `_ + _`. Here, the first underscore represents first input and second underscore represents second input. The only thing to keep in mind here is, if you are using the same input multiple times in body, you won't be able to use this syntax.

In [7]:
List.fold(listInt, 0, _ + _)

cmd7.sc:1: missing parameter type for expanded function ((x$1: <error>, x$2) => x$1.$plus(x$2))
val res7 = List.fold(listInt, 0, _ + _)
                                 ^cmd7.sc:1: missing parameter type for expanded function ((x$1: <error>, x$2: <error>) => x$1.$plus(x$2))
val res7 = List.fold(listInt, 0, _ + _)
                                     ^Compilation Failed

: 

Again, it's complaining about not knowing the type of inputs. We can annotate a type as well.

In [8]:
List.fold(listInt, 0, (_: Int) + (_: Int))

res7: Int = 10

So, placeholder syntax expands as function as follows:

1. `_ + _` is `(a, b) => a + b`
2. `foo(_)` is `x => foo(x)`
3. `bar(_, b)` is `a => bar(a, b)`

#### method to function conversion
---

As we know, method is not a first-class construct in Scala as a function is. But let's say we have some logic implemented as a method and want to pass it as a function to some higher-order method. We don't need to write another function for that, Scala can "convert" that method to function as follows.

If we don't provide any arguments to the method (or provide some of the arguments) then Scala converts it into a function which takes those arguments as a parameter.

In [9]:
object Sum {
    def sum(a: Int, b: Int): Int = a + b
}

Sum.sum _

defined object Sum
res8_1: (Int, Int) => Int = ammonite.$sess.cmd8$Helper$$$Lambda$2580/0x0000000100cec840@490437c4

We didn't provide any arguments to the `sum` method, and Scala returns a function which takes two `Int` arguments and returns an `Int`. We can do the same for partial arguments. For example, rather than not providing both arguments, we provide only `a`. 

In [10]:
Sum.sum(2, _)

res9: Int => Int = ammonite.$sess.cmd9$Helper$$Lambda$2587/0x0000000100cf1840@5e9bef04

This time it returns a function which takes a single argument (in this case `b`) because `a` is already provided.

Note: if we are passing a method directly to a higher-order method, then we can avoid writing `_`.  

In [11]:
List.fold(listInt, 0, Sum.sum)

res10: Int = 10

#### multiple parameter lists
---

As we noticed above, `List.fold(listInt, 0, _ + _)` throws an error because the compiler couldn't figure out the type of function. Even though it knows that `listInt` is `List[Int]` (so `A` is `Int`) and `endCase` is `0` (so `T` is `Int`).

The reason is, it's a limitation of Scala's type inference system, that it can't use type inferred for one parameter for another parameter in the _same_ list.

Scala allows use to define multiple parameter lists for two reasons:
1) at the call-site, method will look like code block (more readable)
2) to assist type system to use inferred type

In [12]:
object List {
    def fold[A, T](list: List[A], endCase: T)(pairCase: (A, T) => T): T = list match {
        case End() => endCase
        case Pair(first, rest) => pairCase(first, fold(rest, endCase)(pairCase))
    }
}

List.fold(listInt, 0)(_ + _)

defined object List
res11_1: Int = 10

This time no errors on using placeholder syntax for function. We can write it as follows as well for more readability.

In [13]:
List.fold(listInt, 0) { (a, b) =>
    // some complicated logic
    a + b
}

res12: Int = 10

### Exercise
---

1. Write a generic method `compose`, which composes any two functions. [hint: `compose(thirdPart, doubleIt)`]

2. Define generic `Tree` data structure, and implement `fold` on it.

3. Use `fold` method to convert `Tree[String]` to `String`. [hint: refer to `List.fold` applied to `listString`]

4. Change `fold` method signature to use multiple argument lists. How does it help?

5. What if you don't provide an argument to the last parameter list of `fold`. Explain.